In [ ]:
import os
import glob
import speech_recognition as sr

from pydub import AudioSegment

audio_dir = "./audio"
extension_list = ('*.m4a', '*.mp3')

os.chdir(audio_dir)

print("Converting files...")

for extension in extension_list:
    
    for audio in glob.glob(extension):
        
        wav_filename = os.path.splitext(os.path.basename(audio))[0] + '.wav'
        AudioSegment.from_file(audio).export(wav_filename, format = 'wav')
        
for filename in os.listdir("."):
    if filename[-3:] != "wav":
        os.remove(filename)

print("Done.")
        
increment = 150

print("Splitting files...")

for filename in os.listdir("."):
    
    audio = AudioSegment.from_wav(filename)
    
    if audio.duration_seconds > increment:
        
        t = 0
        
        while t + increment < audio.duration_seconds:
            
            newAudio = audio[t*1000 : (t + increment)*1000]
            newAudio.export(filename[:-4] + "_" + str(int(t/increment) + 1) + ".wav", format = "wav")
            
            t += increment
        
        newAudio = audio[t*1000 : audio.duration_seconds*1000]
        newAudio.export(filename[:-4] + "_" + str(int(t/increment) + 1) + ".wav", format = "wav")
        
        os.remove(filename)

print("Done.")

sr.__version__
r = sr.Recognizer()
filedict = {}
keylist = []

for filename in os.listdir("."):
    
    print("Processing file " + filename + "...")
    
    harvard = sr.AudioFile(filename)

    with harvard as source:
        audio = r.record(source)

    text = r.recognize_google(audio)
    
    index = filename.find("_")

    if index != -1:
        key = filename[:index]
    else:
        key = filename[:-4]
    
    key = key.replace("media", "slide")
    
    try:
        filedict[key] += " " + text
    except:
        filedict[key] = text
        
    keylist.append(key)

print("Done.")
    
keylist = list(set(keylist))
keylist.sort()    

print("Writing output...")

with open("output.txt", "w", encoding="utf-8") as f:
    for k in keylist:
        f.write(k)
        f.write("\n")
        f.write(filedict[k])
        f.write("\n\n")
        
print("Done.")